In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
import torchvision.models as models


In [10]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

trainset = datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [13]:
class VGG16(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG16, self).__init__()
        self.vgg = models.vgg16(pretrained=False)
        
        self.vgg.features[4] = nn.AvgPool2d(kernel_size=2, stride=2, padding=0, ceil_mode=False)
        self.vgg.features[9] = nn.AvgPool2d(kernel_size=2, stride=2, padding=0,  ceil_mode=False)
        self.vgg.features[16] = nn.AvgPool2d(kernel_size=2, stride=2, padding=0,  ceil_mode=False)
        self.vgg.features[23] = nn.AvgPool2d(kernel_size=2, stride=2, padding=0,  ceil_mode=False)
        self.vgg.features[30] = nn.AvgPool2d(kernel_size=2, stride=2, padding=0,  ceil_mode=False)

        if num_classes == 10:
            self.classifier = nn.Sequential(
                nn.Flatten(),
                nn.Linear(512, 256),
                nn.ReLU(),
                nn.Linear(256, 256),
                nn.ReLU(),
                nn.Linear(256, 10),
            )
        elif num_classes == 200:
            self.classifier = nn.Sequential(
                nn.AvgPool2d(2),
                nn.Flatten(),
                nn.Linear(512, 512),
                nn.ReLU(),
                nn.Linear(512, 512),
                nn.ReLU(),
                nn.Linear(512, 200),
            )
        elif num_classes == 1000:
            self.classifier = nn.Sequential(
                nn.AvgPool2d(2),
                nn.Flatten(),
                nn.Linear(4608, 4096),
                nn.Linear(4096, 4096),
                nn.Linear(4096, 1000)
            )

    def forward(self, x):
        x = self.vgg.features(x)
        x = self.classifier(x)
        return x

class VGG16_Falcon(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG16_Falcon, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(inplace=True),

            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(inplace=True),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(inplace=True),

            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(inplace=True),

            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
        )

        if num_classes == 10:
            self.classifier = nn.Sequential(
                nn.Flatten(),
                nn.Linear(512, 256),
                nn.ReLU(),
                nn.Linear(256, 256),
                nn.ReLU(),
                nn.Linear(256, 10),
            )
        elif num_classes == 200:
            self.classifier = nn.Sequential(
                nn.AvgPool2d(2),
                nn.Flatten(),
                nn.Linear(512, 512),
                nn.ReLU(),
                nn.Linear(512, 512),
                nn.ReLU(),
                nn.Linear(512, 200),
            )
        elif num_classes == 1000:
            self.classifier = nn.Sequential(
                nn.AvgPool2d(2),
                nn.Flatten(),
                nn.Linear(4608, 4096),
                nn.Linear(4096, 4096),
                nn.Linear(4096, 1000)
            )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [14]:
from tensorboardX import SummaryWriter
logger = SummaryWriter(log_dir = 'log')


model = VGG16()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9) 
# VGG16 CryptGPU
# lr=0.001, momentum=0.9 converges about 2~3 epoch. Without momentum, it does not converge.
# lr=0.01 (remove Dropout) converges about 3~4 epoch.

# VGG16 Official
# lr=0.01 

dataiter = iter(trainloader)
# images, labels = dataiter.next()
it = 10
for epoch in range(it):
    for batch, data in enumerate(trainloader, 0):
        images, labels = data
        output = model(images)

        optimizer.zero_grad()
        loss = criterion(output, labels)
        loss.backward()
        
        optimizer.step()
        if batch % 100 == 0:
            print('Epoch: {}, Batch {}, Loss: {}'.format(epoch, batch, loss.item()))

output = model(images)
_, output = torch.max(output, 1)
print(f"Output: ${output}")
print(f"Target: ${labels}")

Epoch: 0, Batch 0, Loss: 2.3094210624694824
Epoch: 0, Batch 100, Loss: 2.297933340072632
Epoch: 0, Batch 200, Loss: 2.304176092147827
Epoch: 0, Batch 300, Loss: 2.304649591445923
Epoch: 0, Batch 400, Loss: 2.3056483268737793
Epoch: 0, Batch 500, Loss: 2.3006937503814697
Epoch: 0, Batch 600, Loss: 2.2962560653686523
Epoch: 0, Batch 700, Loss: 2.297899007797241
Epoch: 0, Batch 800, Loss: 2.3032522201538086
Epoch: 0, Batch 900, Loss: 2.302910327911377
Epoch: 0, Batch 1000, Loss: 2.3034443855285645
Epoch: 0, Batch 1100, Loss: 2.2999305725097656
Epoch: 0, Batch 1200, Loss: 2.3051376342773438
Epoch: 0, Batch 1300, Loss: 2.303309202194214
Epoch: 0, Batch 1400, Loss: 2.300313949584961
Epoch: 0, Batch 1500, Loss: 2.304264545440674
Epoch: 1, Batch 0, Loss: 2.303295850753784
Epoch: 1, Batch 100, Loss: 2.301085948944092
Epoch: 1, Batch 200, Loss: 2.3017776012420654
Epoch: 1, Batch 300, Loss: 2.3042843341827393
Epoch: 1, Batch 400, Loss: 2.3008060455322266
Epoch: 1, Batch 500, Loss: 2.3041262626647